In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

In [3]:
train_path = '../archive/seg_train/'
test_path = '../../blur/data/blur_single/seg_test/'

In [4]:
train_datagen = ImageDataGenerator(
                    rescale = 1/255,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = False)
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (64, 64),
                                                 class_mode = 'categorical')

Found 14034 images belonging to 6 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale = 1/255)
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (64, 64),
                                            class_mode = 'categorical')

Found 3000 images belonging to 6 classes.


In [6]:
cnn = Sequential()
cnn

Metal device set to: Apple M1


2022-08-07 07:57:30.665649: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-07 07:57:30.665800: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Flatten())
cnn.add(Dense(units=256, activation='relu'))
cnn.add(Dense(units=128, activation='relu'))
cnn.add(Dense(units=64, activation='relu'))
cnn.add(Dense(units=24, activation='relu'))
cnn.add(Dense(units=6, activation='softmax'))

In [8]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [9]:
%%time
cnn.fit(training_set, steps_per_epoch= len(training_set), validation_data = test_set, validation_steps=len(test_set), epochs = 17)

Epoch 1/17


2022-08-07 07:57:32.882511: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-07 07:57:33.390800: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


439/439 [==============================] - ETA: 0s - loss: 1.0767 - accuracy: 0.5822

2022-08-07 07:57:58.519487: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


439/439 [==============================] - 28s 55ms/step - loss: 1.0767 - accuracy: 0.5822 - val_loss: 1.1384 - val_accuracy: 0.5490
Epoch 2/17
439/439 [==============================] - 22s 50ms/step - loss: 0.7899 - accuracy: 0.7092 - val_loss: 1.6121 - val_accuracy: 0.4680
Epoch 3/17
439/439 [==============================] - 21s 49ms/step - loss: 0.6776 - accuracy: 0.7545 - val_loss: 1.1965 - val_accuracy: 0.6037
Epoch 4/17
439/439 [==============================] - 21s 48ms/step - loss: 0.6157 - accuracy: 0.7767 - val_loss: 1.3655 - val_accuracy: 0.5750
Epoch 5/17
439/439 [==============================] - 21s 48ms/step - loss: 0.5876 - accuracy: 0.7894 - val_loss: 1.3182 - val_accuracy: 0.5893
Epoch 6/17
439/439 [==============================] - 23s 52ms/step - loss: 0.5241 - accuracy: 0.8117 - val_loss: 1.1378 - val_accuracy: 0.6557
Epoch 7/17
439/439 [==============================] - 23s 52ms/step - loss: 0.5189 - accuracy: 0.8166 - val_loss: 1.2047 - val_accuracy: 0.5837
Epo

In [9]:
accuracy=cnn.evaluate_generator(test_set, steps=len(test_set)//32,verbose=2)
print('Accuracy of the model on the test set: ',accuracy[1])